In [7]:
import tensorflow as tf
import numpy as np
import pickle

with open('../data/conv_lstm_sequence_data7.pc', 'rb') as f:
    raw_data = pickle.load(f)
# raw_data = np.expand_dims(raw_data, axis=-1)
print("raw dataset shape: {}".format(raw_data.shape))


raw dataset shape: (52008, 20, 30, 1)


In [2]:
### THESE CODES ARE MEANT FOR TF2

In [8]:
raw_data = raw_data / 2
train, test = raw_data[:48000], raw_data[48000:]
print("train, test shapes: {}-{}".format(train.shape, test.shape))

train, test shapes: (48000, 20, 30, 1)-(4008, 20, 30, 1)


In [ ]:
from tensorflow.data import Dataset

seq_len = 18 # it's 1 hour and a half
batch_size = 10

train_ds = Dataset.from_tensor_slices(train)
test_ds = Dataset.from_tensor_slices(test)

test_seq = train_ds.batch(seq_len+1, drop_remainder=True)
sequences = train_ds.batch(seq_len+1, drop_remainder=True)

test_ds = test_seq.map(lambda x: (x[:-1], x[1:]))
test_ds = test_ds.batch(batch_size, drop_remainder=True)
ds = sequences.map(lambda x: (x[:-1], x[1:]))
ds = ds.shuffle(10000).batch(batch_size, drop_remainder=True)

In [ ]:
ds

In [ ]:
from tensorflow.keras.models import Sequential

model = Sequential([
    tf.keras.layers.ConvLSTM2D(filters=20, kernel_size=(3,3), padding='same', input_shape=(None, 30, 40, 1), return_sequences=True),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ConvLSTM2D(filters=20, kernel_size=(3,3), padding='same', return_sequences=True),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv3D(filters=1, kernel_size=(3,3,3), activation='sigmoid', padding='same')
])

model.compile(loss='mse', optimizer='adam')
model.summary()

In [ ]:
model.fit(ds, epochs=5, validation_data=test_ds)

In [26]:
model.evaluate(test_ds)

78/78 [==============================] - 52s 661ms/step - loss: 0.3015


0.3014840982281245

# COMPARE

in this section I compare it with return_sequence=False

In [9]:
from tensorflow.data import Dataset

seq_len = 19
batch_size = 20

train_ds = Dataset.from_tensor_slices(train)
test_ds = Dataset.from_tensor_slices(test)

test_seq = train_ds.batch(seq_len+1, drop_remainder=True)
sequences = train_ds.batch(seq_len+1, drop_remainder=True)

test_ds = test_seq.map(lambda x: (x[:-1], x[-1]))
test_ds = test_ds.batch(batch_size, drop_remainder=True)
ds = sequences.map(lambda x: (x[:-1], x[-1]))
ds = ds.shuffle(10000).batch(batch_size, drop_remainder=True)

ds

<BatchDataset shapes: ((20, 19, 20, 30, 1), (20, 20, 30, 1)), types: (tf.float64, tf.float64)>

In [11]:
from tensorflow.keras.models import Sequential

model = Sequential([
    tf.keras.layers.ConvLSTM2D(filters=5, kernel_size=(5,5), padding='same', input_shape=(None, 20, 30, 1), return_sequences=True),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ConvLSTM2D(filters=10, kernel_size=(3,3), padding='same', return_sequences=True),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ConvLSTM2D(filters=15, kernel_size=(2,2), padding='same', return_sequences=True),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ConvLSTM2D(filters=20, kernel_size=(3,3), padding='same', return_sequences=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=1, kernel_size=(1,1), padding='same'),
    tf.keras.layers.Dropout(0.2)
])

model.compile(loss='mse', optimizer='adam')
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_12 (ConvLSTM2D) (None, None, 20, 30, 5)   3020      
_________________________________________________________________
batch_normalization_12 (Batc (None, None, 20, 30, 5)   20        
_________________________________________________________________
conv_lst_m2d_13 (ConvLSTM2D) (None, None, 20, 30, 10)  5440      
_________________________________________________________________
batch_normalization_13 (Batc (None, None, 20, 30, 10)  40        
_________________________________________________________________
conv_lst_m2d_14 (ConvLSTM2D) (None, None, 20, 30, 15)  6060      
_________________________________________________________________
batch_normalization_14 (Batc (None, None, 20, 30, 15)  60        
_________________________________________________________________
conv_lst_m2d_15 (ConvLSTM2D) (None, 20, 30, 20)       

In [12]:
model.fit(ds, epochs=40, validation_data=test_ds)

Epoch 1/40


W0720 17:57:00.531263 140461164345152 deprecation.py:323] From /home/navid/venvs/tf2env/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


120/120 [==============================] - 390s 3s/step - loss: 0.8663 - val_loss: 1.1259
Epoch 2/40
120/120 [==============================] - 379s 3s/step - loss: 0.6601 - val_loss: 0.8786
Epoch 3/40
120/120 [==============================] - 391s 3s/step - loss: 0.6369 - val_loss: 0.6821
Epoch 4/40
120/120 [==============================] - 422s 4s/step - loss: 0.6299 - val_loss: 0.5661
Epoch 5/40
120/120 [==============================] - 417s 3s/step - loss: 0.6211 - val_loss: 0.5250
Epoch 6/40
120/120 [==============================] - 400s 3s/step - loss: 0.6204 - val_loss: 0.4602
Epoch 7/40
120/120 [==============================] - 398s 3s/step - loss: 0.6160 - val_loss: 0.4266
Epoch 8/40
 55/120 [============>.................] - ETA: 2:45 - loss: 0.6229

KeyboardInterrupt: 

In [ ]:
model.evaluate(test_ds)

116/120 [============================>.] - ETA: 3s - loss: 0.4323

In [ ]:
model.save('trained_on_6month_7epochs.h5')

In [ ]:
### THESE CODES ARE MEANT FOR TF1.14

In [ ]:
from keras.models import Sequential
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
import numpy as np
import pylab as plt

# We create a layer which take as input movies of shape
# (n_frames, width, height, channels) and returns a movie
# of identical shape.

with tf.device('/cpu:0'):
    model = Sequential()
    model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                       input_shape=(None, 30, 40, 1),
                       padding='same', return_sequences=True))
    model.add(BatchNormalization())

    model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                       padding='same', return_sequences=True))
    model.add(BatchNormalization())

    model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                       padding='same', return_sequences=True))
    model.add(BatchNormalization())

    model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                       padding='same', return_sequences=True))
    model.add(BatchNormalization())

    model.add(Conv3D(filters=1, kernel_size=(3, 3, 3),
                   activation='sigmoid',
                   padding='same', data_format='channels_last'))
    model.compile(loss='binary_crossentropy', optimizer='adadelta')

model.summary()

In [ ]:
print(raw_data.shape)
seq_len = 18
bs = 10

remainder = raw_data.shape[0]%seq_len
raw_data = raw_data[:raw_data.shape[0]-remainder,:]

print(raw_data.shape)

seq = np.array(np.split(raw_data, raw_data.shape[0]//seq_len))

In [ ]:
input_data, target = seq[:,:-1, :], seq[:, 1:, :]
print(input_data.shape, target.shape)


In [ ]:
model.fit(input_data, target, batch_size=10, epochs=10, validation_split=0.05)

In [ ]:
from tensorflow.data import Dataset

seq_len = 18 # it's 1 hour and a half
batch_size = 5

ds = Dataset.from_tensor_slices(raw_data)
sequences = ds.batch(seq_len+1, drop_remainder=True).shuffle(10000).batch(batch_size, drop_remainder=True)

ds = sequences.map(lambda x: x[:,:-1])
target = sequences.map(lambda x: x[:,1:])


In [ ]:
ds, target

In [ ]:
seq.fit(ds, target, steps_per_epoch=10)